In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
from api_keys import base_url

In [2]:
#Load the csv exported in Part I to a DataFrame
intial_df = pd.read_csv("output_data/cities.csv")
intial_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [3]:
# Humidity Heatmap
#Configure gmaps.
gmaps.configure(api_key=g_key)
#Use the Lat and Lng as locations and Humidity as the weight.
locations = intial_df[["Lat","Lng"]]
weight = intial_df["Humidity"]
#Plot 1st map
gmap = gmaps.figure(center = [0,0], zoom_level=2)

#Add Heatmap layer to map.
hmlayer = gmaps.heatmap_layer(locations,weight)
hmlayer.point_radius = 4
hmlayer.max_intensity = 200
hmlayer.dissipating = False
                   
gmap.add_layer(hmlayer)
#Show Weather map
gmap

Figure(layout=FigureLayout(height='420px'))

In [4]:
#Create new DataFrame fitting weather criteria
#Narrow down the cities to fit weather conditions.
wcdf = intial_df.loc[(intial_df["Max Temp"] <= 80) & (intial_df["Max Temp"] >= 70)]
wcdf = wcdf.loc[(wcdf["Wind Speed"] <=10) & (wcdf["Cloudiness"]==0)]

#Drop any rows will null values.
wcdf = wcdf.dropna()
wcdf

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


In [5]:
# Hotel Map

#Store into variable named hotel_df.
hotel_df = wcdf

#Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = np.nan
#Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}
#Hit the Google Places API for each city's coordinates.
for  index, row in hotel_df.iterrows(): 
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # add/modify location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    #  API request
    response = requests.get(base_url, params=params).json()
    results = response['results']
    #Store the first Hotel result into the DataFrame.
    hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
hotel_df


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Hotel Escarpas do Lago
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Castelo
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,La Case Bambou
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Dandeli Lake County
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Airport hotel
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,MOGOK HILL HOTEL
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,تاکسی سرویس آرامش


In [7]:
#Plot markers on top of the heatmap.
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

# Display figure
gmap.add_layer(markers)
gmap.add_layer(hotel_layer)
gmap

Figure(layout=FigureLayout(height='420px'))